# TextMining_messy_medical_dates


In this script, we will handle complex medical data where dates are encoded in various formats using regular expressions (regex) to extract relevant information.

Each line in the `dates.txt` file corresponds to a medical note, and each note contains a date that needs to be extracted. However, dates appear in many different formats.

The objective of this script is to accurately identify all the different date formats present in the dataset, normalize them, and sort them properly.

Here are examples of the date variants that may be encountered:

* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once dates are extracted from the text using these patterns, the next step is to sort them chronologically in ascending order according to specific rules:

* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

In [19]:
import numpy as np
import pandas as pd

def date_sorter():
    # Cargar el archivo de texto como una Serie de pandas
    doc = []
    with open('assets/dates.txt') as file:
        for line in file:
            doc.append(line)

    df = pd.Series(doc)

    # Extraer todas las variantes de fechas con expresiones regulares
    # Variantes de fechas del tipo xx/xx/xx o xx/xx/xxxx
    a1_1 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{2})\b')
    a1_2 = df.str.extractall(r'(\d{1,2})[/-](\d{1,2})[/-](\d{4})\b')
    a1 = pd.concat([a1_1, a1_2])
    a1.reset_index(inplace=True)
    a1_index = a1['level_0']

    # Variantes de fechas del tipo "Month Day, Year" o "Month. Day, Year"
    a2 = df.str.extractall(r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-.]* )((?:\d{1,2}[?:, -]*)\d{4})')
    a2.reset_index(inplace=True)
    a2_index = a2['level_0']

    # Variantes de fechas del tipo "Day Month Year" o "Day Month, Year"
    a3 = df.str.extractall(r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})')
    a3.reset_index(inplace=True)
    a3_index = a3['level_0']

    # Variantes de fechas del tipo xx/xxxx
    a6 = df.str.extractall(r'(\d{1,2})[/](\d{4})')
    a6.reset_index(inplace=True)
    a6_index = a6['level_0']

    # Filtrar variantes a6 que no estén en a1 (formatos xx/xx/xx o xx/xx/xxxx)
    save = []
    for i in a6_index:
        if not (i in a1_index.values):
            save.append(i)
    save = np.asarray(save)
    a6 = a6[a6['level_0'].isin(save)]

    # Variantes de fechas del tipo "Year" o "Year"
    a7_1 = df.str.extractall(r'[a-z]?[^0-9](\d{4})[^0-9]')
    a7_2 = df.str.extractall(r'^(\d{4})[^0-9]')
    a7 = pd.concat([a7_1, a7_2])
    a7.reset_index(inplace=True)
    a7_index = a7['level_0']

    # Filtrar las fechas en a7 para asegurarse de que no se superpongan con las fechas ya extraídas en a2, a3, y a6.
    save = []
    for i in a7_index:
        if not ((i in a2_index.values) | (i in a3_index.values) | (i in a6_index.values)):
            save.append(i)
    save = np.asarray(save)
    a7 = a7[a7['level_0'].isin(save)]

    # Crear un único arreglo que contenga todos los índices de las filas del DataFrame original (df)
    s = np.asarray(a1.level_0.values.tolist() + a2.level_0.values.tolist() +
                   a3.level_0.values.tolist() + a6.level_0.values.tolist() +
                   a7.level_0.values.tolist())

    # Procesar cada DataFrame resultante para normalizar las fechas
    a1.columns = ['level_0', 'match', 'month', 'day', 'year']
    a1['year'] = a1['year'].apply(str)
    a1['year'] = a1['year'].apply(lambda x: '19'+x if len(x) <= 2 else x)

    a2[1] = a2[1].apply(lambda x: x.replace(',', ''))
    a2['day'] = a2[1].apply(lambda x: x.split(' ')[0])
    a2['year'] = a2[1].apply(lambda x: x.split(' ')[1])
    a2.columns = ['level_0', 'match', 'month', 'day-year', 'day', 'year']
    a2.drop('day-year', axis=1, inplace=True)

    a3.columns = ['level_0', 'match', 'day', 'month', 'year']
    a3['day'] = a3['day'].replace(np.nan, -99)
    a3['day'] = a3['day'].apply(lambda x: 1 if int(x) == -99 else x)
    a3['month'] = a3.month.apply(lambda x: x[:3])
    a3['month'] = pd.to_datetime(a3.month, format='%b').dt.month

    a6.columns = ['level_0', 'match', 'month', 'year']
    a6['day'] = 1

    a7.columns = ['level_0', 'match', 'year']
    a7['day'] = 1
    a7['month'] = 1

    # Concatenar todos los resultados procesados en un DataFrame final
    final = pd.concat([a1, a2, a3, a6, a7])

    # Combinar las columnas 'month', 'day', y 'year' en una nueva columna 'date'
    # errors='coerce': Este argumento indica que si una cadena no puede convertirse en una fecha válida, en lugar de generar un error, se asignará NaT (Not a Time) a esa entrada.
    final['date'] = pd.to_datetime(final['month'].apply(str) + '/' + final['day'].apply(str) + '/' + final['year'].apply(str), errors='coerce')

    # Ordenar el DataFrame final por el índice original y establecerlo como índice
    final = final.sort_values(by='level_0').set_index('level_0')

    # Obtener la lista de fechas ordenadas
    DateList = final['date']

    return DateList

# Llamar a la función para ejecutarla
date_sorter()

level_0
0     1993-03-25
1     1985-06-18
2     1971-07-08
3     1975-09-27
4     1996-02-06
         ...    
495   1979-01-01
496   2006-01-01
497   2008-01-01
498   2005-01-01
499   1980-01-01
Name: date, Length: 500, dtype: datetime64[ns]